---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [78]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [39]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [79]:
#    '''Returns a DataFrame of towns and the states they are in from the 
#    university_towns.txt list. The format of the DataFrame should be:
#    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
#    columns=["State", "RegionName"]  )
    
#    The following cleaning needs to be done:

#    1. For "State", removing characters from "[" to the end.
#    2. For "RegionName", when applicable, removing every character from " (" to the end.
#    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

def get_list_of_university_towns():
    uni_towns = pd.read_csv('university_towns.txt', error_bad_lines=False, names=['RegionName'])

    uni_towns['State'] = pd.np.where(uni_towns.RegionName.str.contains('edit'), uni_towns['RegionName'], '')
    uni_towns['RegionName'] = uni_towns[~uni_towns.RegionName.str.contains("edit")]

    uni_towns['State'] = uni_towns.State.replace(r'^\s*$', np.NaN, regex=True)
    uni_towns['State'] = uni_towns['State'].str.replace("\[.*\]", '')
    uni_towns['State'] = uni_towns.State.fillna(method='ffill')

    uni_towns['RegionName'] = uni_towns['RegionName'].str.replace("\[.*\]", '')
    uni_towns['RegionName'] = uni_towns['RegionName'].str.replace("\(.*\)", '')
    uni_towns['RegionName'] = uni_towns['RegionName'].str.replace("\(.*", '')
    uni_towns['RegionName'] = uni_towns['RegionName'].str.strip()

    uni_towns = uni_towns[['State', 'RegionName']]
    uni_towns = uni_towns.dropna()
    
    return uni_towns

In [80]:
#'''Returns the year and quarter of the recession start time as a 
#string value in a format such as 2005q3'''

def get_recession_start():
    
    rec_inf = pd.read_excel('gdplev.xls', skiprows=5)

    rec_inf.drop(rec_inf.iloc[:, 0:4], axis=1, inplace=True)
    rec_inf.drop(rec_inf.columns[[1, 3]], axis=1, inplace=True)
    rec_inf.drop(rec_inf.index[:214], inplace=True)

    rec_inf.reset_index(drop=True, inplace=True)
    rec_inf.columns = ['Year_Q', 'GDP_Billions']

    occurrences = 0
    i = 0
    answer = ''
    while(occurrences != 2):
        if rec_inf.iloc[i+1]['GDP_Billions'] < rec_inf.iloc[i]['GDP_Billions']:
            occurrences = occurrences + 1
            i = i + 1
            if occurrences == 2:
                answer = rec_inf.iloc[i-1]['Year_Q']
        else:
            if occurrences == 1:
                occurrences = 0
            i = i + 1
    
    return answer

In [81]:
#'''Returns the year and quarter of the recession end time as a 
#string value in a format such as 2005q3'''

def get_recession_end():
    
    rec_inf = pd.read_excel('gdplev.xls', skiprows=5)

    rec_inf.drop(rec_inf.iloc[:, 0:4], axis=1, inplace=True)
    rec_inf.drop(rec_inf.columns[[1, 3]], axis=1, inplace=True)
    rec_inf.drop(rec_inf.index[:248], inplace=True)

    rec_inf.reset_index(drop=True, inplace=True)
    rec_inf.columns = ['Year_Q', 'GDP_Billions']

    occurrences = 0
    i = 0
    answer = ''
    while(occurrences != 2):
        if rec_inf.iloc[i+1]['GDP_Billions'] > rec_inf.iloc[i]['GDP_Billions']:
            occurrences = occurrences + 1
            i = i + 1
            if occurrences == 2:
                answer = rec_inf.iloc[i]['Year_Q']
        else:
            if occurrences == 1:
                occurrences = 0
            i = i + 1
      
    return answer

In [82]:
#'''Returns the year and quarter of the recession bottom time as a 
#string value in a format such as 2005q3'''

def get_recession_bottom():
    
    rec_inf = pd.read_excel('gdplev.xls', skiprows=5)

    rec_inf.drop(rec_inf.iloc[:, 0:4], axis=1, inplace=True)
    rec_inf.drop(rec_inf.columns[[1, 3]], axis=1, inplace=True)
    rec_inf.drop(rec_inf.index[:248], inplace=True)
    rec_inf.drop(rec_inf.index[6:], inplace=True)

    rec_inf.reset_index(drop=True, inplace=True)
    rec_inf.columns = ['Year_Q', 'GDP_Billions']
    rec_inf = rec_inf.set_index('Year_Q')
    answer = ''.join(rec_inf[rec_inf['GDP_Billions'] == rec_inf['GDP_Billions'].min()].index.values)
    
    return answer

In [83]:
#'''Converts the housing data to quarters and returns it as mean 
#values in a dataframe. This dataframe should be a dataframe with
#columns for 2000q1 through 2016q3, and should have a multi-index
#in the shape of ["State","RegionName"].
    
#Note: Quarters are defined in the assignment description, they are
#not arbitrary three month periods.
    
#The resulting dataframe should have 67 columns, and 10,730 rows.
#'''


def convert_housing_data_to_quarters():
    
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    housing_data.drop(housing_data.iloc[:, 6:51], inplace=True, axis=1)

    quarters_data = housing_data.iloc[:, 6:]
    quarters_data.columns = pd.to_datetime(quarters_data.columns)
    quarters_data = quarters_data.resample('Q', axis=1).mean()
    quarters_data = quarters_data.rename(columns=lambda col: '{}q{}'.format(col.year, col.quarter))

    housing_data.drop(housing_data.iloc[:, 6:], inplace=True, axis=1)

    join_df = pd.concat([housing_data, quarters_data], axis=1)

    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

    join_df['States'] = join_df['State'].map(states)
    join_df.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank', 'State'], axis=1, inplace=True)
    join_df.rename(columns={'States':'State'}, inplace=True)
    join_df.sort_values(by=['State', 'RegionName'], ascending=True, inplace=True)
    join_df.set_index(["State","RegionName"], inplace=True)
    
    return join_df

In [104]:
#'''First creates new data showing the decline or growth of housing prices
#between the recession start and the recession bottom. Then runs a ttest
#comparing the university town values to the non-university towns values, 
#return whether the alternative hypothesis (that the two groups are the same)
#is true or not as well as the p-value of the confidence. 
    
#Return the tuple (different, p, better) where different=True if the t-test is
#True at a p<0.01 (we reject the null hypothesis), or different=False if 
#otherwise (we cannot reject the null hypothesis). The variable p should
#be equal to the exact p value returned from scipy.stats.ttest_ind(). The
#value for better should be either "university town" or "non-university town"
#depending on which has a lower mean price ratio (which is equivilent to a
#reduced market loss).'''


def run_ttest():
    
    univ_town_data = get_list_of_university_towns()
    univ_town_data['univ_town'] = True

    HD_UD_Join = pd.merge(convert_housing_data_to_quarters(), univ_town_data, how='outer', left_index=True, right_on=['State', 'RegionName'])
    HD_UD_Join['univ_town'] = HD_UD_Join['univ_town'].replace({np.NaN: False})

    HD_UD_Join.drop(HD_UD_Join.iloc[:, :34], inplace = True, axis = 1)
    HD_UD_Join.drop(HD_UD_Join.iloc[:, 4:33], inplace = True, axis = 1)

    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()

    new_columns_order = ['State', 'RegionName', 'univ_town', '2008q3', '2008q4', '2009q1', '2009q2']
    HD_UD_Join = HD_UD_Join.reindex(columns=new_columns_order)

    HD_UD_Join['price_ratio'] = HD_UD_Join[recession_bottom] - HD_UD_Join[recession_start]

    univ_town_price_ratio = HD_UD_Join[HD_UD_Join['univ_town'] == True]['price_ratio']
    non_univ_town_price_ratio = HD_UD_Join[HD_UD_Join['univ_town'] == False]['price_ratio']

    st, p = ttest_ind(univ_town_price_ratio, non_univ_town_price_ratio, nan_policy='omit',)

    different = False
    if p < 0.01:
        different = True

    better = ""
    if univ_town_price_ratio.mean() > non_univ_town_price_ratio.mean():
        better = "university town"
    else:
        better = "non-university town"
    
    return (different, p, better)

(True, 0.0043883080860228811, 'university town')